In [10]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [11]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-12 01:11:44--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.67MB/s    in 0.2s    

2022-06-12 01:11:44 (5.67 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [12]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [13]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [14]:
# Drop duplicates and incomplete rows
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

4880466
4879961
4879961


In [15]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1002B10PD9ZFM|          5|            0|          0|   N|                Y|
|R1007DQP1BG5WJ|          5|            0|          0|   N|                Y|
|R100AIJ0ZFN8CZ|          4|            3|          3|   N|                N|
|R100C48B994FWD|          5|            0|          0|   N|                Y|
|R100E300CKVFYD|          1|            3|          3|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [72]:
# Step 1: Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes 
# count is = to or greater than 20.
from pyspark.sql.functions import col
total_votes_df = vine_df.filter(col("total_votes") >= 20)
total_votes_df.show(5)
print(total_votes_df.count())

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1TKZ614B5ZRAD|          4|           31|         32|   N|                Y|
|R2KSNXQSNNISFA|          5|           20|         20|   N|                Y|
|R1TNMKO6WINLZ2|          1|           53|         58|   N|                Y|
| R1FXSTWOLWAKI|          1|           35|         38|   N|                Y|
|R11RPG1PWD5JRN|          1|           23|         25|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows

107390


In [73]:
# Step 2: Filter the new DataFrame and create a new DataFrame or table to retrieve all the rows where the
# number of helpful_votes / by total_votes is = to or greater than 50%.
percent_votes_df = total_votes_df.withColumn('percent_votes', col('helpful_votes') / col('total_votes')).alias('percent_votes').filter(col('percent_votes') >= 0.5)
percent_votes_df.show(5)
print(percent_votes_df.count())

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1TKZ614B5ZRAD|          4|           31|         32|   N|                Y|           0.96875|
|R2KSNXQSNNISFA|          5|           20|         20|   N|                Y|               1.0|
|R1TNMKO6WINLZ2|          1|           53|         58|   N|                Y|0.9137931034482759|
| R1FXSTWOLWAKI|          1|           35|         38|   N|                Y|0.9210526315789473|
|R11RPG1PWD5JRN|          1|           23|         25|   N|                Y|              0.92|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
only showing top 5 rows

99017


In [23]:
# Step 3: Filter the DataFrame and create a new DataFrame or table that retrieves all the rows where a review
# was written as par tof the Vine program (paid), vine == 'Y'.
paid_df = percent_votes_df.filter(col("vine") == 'Y')
paid_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RWWAEKASA5598|          3|           35|         42|   Y|                N|0.8333333333333334|
|R3Q209RODQNKB3|          5|           41|         44|   Y|                N|0.9318181818181818|
|R3G2UY1TPJQ21J|          4|           37|         39|   Y|                N|0.9487179487179487|
|R1F3OMRBC0FAB9|          5|           25|         28|   Y|                N|0.8928571428571429|
|R3AOXRAS5ZEMFK|          5|           20|         21|   Y|                N|0.9523809523809523|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
only showing top 5 rows



In [24]:
# Step 4: Filter the DataFrame and create a new DataFrame or table that retrieves all the rows where a review
# was written as par tof the Vine program (paid), vine == 'N'.
unpaid_df = percent_votes_df.filter(col("vine") == 'N')
unpaid_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1TKZ614B5ZRAD|          4|           31|         32|   N|                Y|           0.96875|
|R2KSNXQSNNISFA|          5|           20|         20|   N|                Y|               1.0|
|R1TNMKO6WINLZ2|          1|           53|         58|   N|                Y|0.9137931034482759|
| R1FXSTWOLWAKI|          1|           35|         38|   N|                Y|0.9210526315789473|
|R11RPG1PWD5JRN|          1|           23|         25|   N|                Y|              0.92|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
only showing top 5 rows



In [74]:
# Step 5: Determine the total # of reviews, # of 5-star reviews, % of 5-star reviews for the two types of reviews (paid vs. unpaid).
total_reviews_df = percent_votes_df.groupBy("vine").agg(count(col("vine")).alias("Number_of_Total_Reviews"), count(when(col("star_rating") == 5, True)).alias("Number_of_5_Star_Ratings"), (count(when(col("star_rating") == 5, True))/count(col("vine"))*100).alias("%_of 5_Star_Ratings_vs_Total")).show()

+----+-----------------------+------------------------+----------------------------+
|vine|Number_of_Total_Reviews|Number_of_5_Star_Ratings|%_of 5_Star_Ratings_vs_Total|
+----+-----------------------+------------------------+----------------------------+
|   Y|                   1207|                     509|           42.17067108533554|
|   N|                  97810|                   45846|          46.872507923525205|
+----+-----------------------+------------------------+----------------------------+

